In [1]:
import tracemalloc

from utils_v1 import *
import traceback


In [ ]:
from icecream import ic

import numpy as np
from src.dataset import (
    MultipleDates, PAMultipleDates,
    MTMultipleDates, MSMultipleDates, PIMultipleDates
)

# from src.dataset_legacy import MTDeforestationTime
from src import training_generator as generator
from src.patchesHandler import PatchesHandler, PatchesHandlerMultipleDates, PatchesHandlerEvidential2

from src.manager.evidential import ManagerEvidential
from src.manager.multioutput import ManagerMCDropout, ManagerSingleRun, ManagerEvidential2
from mpl_toolkits.axes_grid1 import make_axes_locatable

import cv2
from src import metrics as _metric
import json
import pickle 
import src.manager.evidential as evidential
from src.Logger import Logger
import pdb
import gc


Tensorflow ver. 2.6.0


In [ ]:

with open('config/config.json', 'r') as f:
    config = json.load(f)
    
print("Mode: {}. Site: {}. Training: {}. Training date: {}".format(
    config['mode'], config['site'], config['training'], config['training_date']))
print(config)


Mode: evidential2. Site: MS. Training: True. Training date: current
{'training': True, 'inferring': True, 'site': 'MS', 'training_date': 'current', 'mode': 'evidential2', 'training_times': 20, 'uncertainty_method': 'pred_entropy', 'loadInference': False, 'removePolygons': True, 'plotLandsat': False, 'plotPOI': False, 'training_experiment_id': 41, 'get_multiple_metrics': False, 'start_repetition_id': 0, 'end_repetition_id': 20, 'save_probabilities': False, 'addPastDeforestationInput': True, 'classes_mode': True, 'learning_rate': 0.0001, 'use_cloud_mask': False, 'mask_out_non_biome': False, 'log_filename': 'log.pkl'}


In [ ]:

if config['site'] == 'PA':
	if config['training_date'] == 'earlier':
		dates = [2017, 2018]
	else:
		dates = [2018, 2019]
	datasetClass = PAMultipleDates
elif config['site'] == 'MT':
	if config['training_date'] == 'earlier':
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]
	datasetClass = MTMultipleDates
elif config['site'] == 'MS':
	if config['training_date'] == 'earlier':
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]
	datasetClass = MSMultipleDates
elif config['site'] == 'PI':
	if config['training_date'] == 'earlier':
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]
	datasetClass = PIMultipleDates
else:
	raise ValueError('Invalid site')

dataset = datasetClass(config = config,
			   dates = dates, 
	 	addPastDeforestationInput = config['addPastDeforestationInput'],
		borderBuffer = 2)	

if config['mode'] == 'ensemble':
	if config['training'] == True:
		config['mode'] = 'single_run'
	else:
		sys.exit("Ensure that training is True for ensemble mode. Use infer_ensemble.ipynb for inference.")

if config['training'] == False:
	if config['site'] == 'PA':
		if config['mode'] == 'mcd' or config['mode'] == 'single_run': # MCD will do 10 inference on each of the 10 repetitions = 100 runs
			if config['training_date'] == 'current': exp = 0
			elif config['training_date'] == 'earlier': exp = 1
				
		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 8; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 9; repetition_id = 0

	elif config['site'] == 'MT':

		if config['mode'] == 'mcd' or config['mode'] == 'single_run':
			if config['training_date'] == 'current': exp = 2
			elif config['training_date'] == 'earlier': exp = 3
		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 10; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 11; repetition_id = 0

	elif config['site'] == 'MS':

		if config['mode'] == 'mcd' or config['mode'] == 'single_run':
			if config['training_date'] == 'current': exp = 40
			elif config['training_date'] == 'earlier': 
				exp = 37

		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 12; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 39; repetition_id = 0
	elif config['site'] == 'PI':

		if config['mode'] == 'mcd' or config['mode'] == 'single_run':
			if config['training_date'] == 'current': exp = 6
			elif config['training_date'] == 'earlier': exp = 27
		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 14; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 39; repetition_id = 0

else:
	exp = config['training_experiment_id']
ic(dates)


if issubclass(type(dataset), MultipleDates):
	patchesHandler = PatchesHandlerMultipleDates(
		dataset)
elif config['mode'] == 'evidential2':
	patchesHandler = PatchesHandlerEvidential2()
else:
	patchesHandler = PatchesHandler()



ic| list(self.date_ids): [0]
ic| self.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8]]
ic| dates: [2019, 2020]
ic| self.input_image_shape: 9
ic| self.dataset.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8]]


In [ ]:


if config['mode'] == 'evidential2':
    config["inference_times"] = 1
    manager_class = ManagerEvidential2
    config['uncertainty_method'] = "pred_entropy"
    config['classes_mode'] = True
    
if config['mode'] == 'mcd':
    config["inference_times"] = 10
    manager_class = ManagerMCDropout
    config['uncertainty_method'] = "pred_entropy"
    config['classes_mode'] = False

elif config['mode'] == 'single_run':
    config["inference_times"] = 1
    manager_class = ManagerSingleRun
    config['uncertainty_method'] = "pred_entropy_single"
    config['classes_mode'] = False



In [ ]:
exp

41

In [ ]:
logger = Logger()


In [ ]:
if config['training'] == True:
    

    manager = manager_class(config, dataset, patchesHandler, logger)
    print("manager.config['dropout_training']", manager.config['dropout_training'])
    manager.defineExperiment(exp) # fixed
    
    manager.setExperimentPath()
    manager.createLogFolders()

    manager.loadDataset()

    # %%
    if config["training"] == True:
        manager.run()


manager.config['dropout_training'] False
D:/Jorge/datasets/deforestation/MS/deforestation_past_years.tif
Loaded deforestation past years


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2002, 2004, 2006, 2008, 2010, 2012, 2013, 2014, 2015, 2016,
                                                                    2017, 2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([334024764,   8717093,   6541094,   3824922,   2096535,   1305948,
                                                                      1003721,    903906,    318788,    701575,    310887,    440417,
                                                                       316180,    560438,    600838,    776899], dtype=int64))


Label where deforestation past years is actual date (2020) = 1


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2002, 2004, 2006, 2008, 2010, 2012, 2013, 2014, 2015, 2016,
                                                                    2017, 2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([334024764,   8717093,   6541094,   3824922,   2096535,   1305948,
                                                                      1003721,    903906,    318788,    701575,    310887,    440417,
                                                                       316180,    560438,    600838,    776899], dtype=int64))


Past deforestation different from 0 (no deforestation)


...

training samples:  (1660, 3) validation samples:  (277, 3)
time:  0


c:\Users\jchamorro\Anaconda3\envs\tf2\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_enc_net (InputLayer)      [(None, 128, 128, 9) 0                                            
__________________________________________________________________________________________________
res1_net1 (Conv2D)              (None, 128, 128, 16) 1312        input_enc_net[0][0]              
__________________________________________________________________________________________________
drop_net1 (SpatialDropout2D)    (None, 128, 128, 16) 0           res1_net1[0][0]                  
__________________________________________________________________________________________________
res2_net1 (Conv2D)              (None, 128, 128, 16) 2320        drop_net1[0][0]                  
______________________________________________________________________________________________

c:\Users\jchamorro\Anaconda3\envs\tf2\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


155/155 [==============================] - 7s 46ms/step - loss: 0.2911 - accuracy: 0.3715 - val_loss: 0.2708 - val_accuracy: 0.3434

Epoch 00001: val_loss did not improve from 0.26925
Early stop count: 1
Epoch: 3
Anneling Coeficient [0.2]
155/155 [==============================] - 7s 46ms/step - loss: 0.2911 - accuracy: 0.3719 - val_loss: 0.2733 - val_accuracy: 0.3507

Epoch 00001: val_loss did not improve from 0.26925
Early stop count: 2
Epoch: 4
Anneling Coeficient [0.3]
155/155 [==============================] - 7s 46ms/step - loss: 0.2911 - accuracy: 0.3721 - val_loss: 0.2678 - val_accuracy: 0.3362

Epoch 00001: val_loss improved from 0.26925 to 0.26778, saving model to D:/Jorge/datasets/deforestation/experiments/MS/exp41/models\resunet_0.h5
New best val loss. Val loss: 0.26778. Early stop count: 0
Epoch: 5
Anneling Coeficient [0.4]
155/155 [==============================] - 7s 46ms/step - loss: 0.2910 - accuracy: 0.3721 - val_loss: 0.2672 - val_accuracy: 0.3306

Epoch 00001: val_l

In [ ]:



if config['site'] == 'PA':
	dates = [2018, 2019]
	datasetClass = PAMultipleDates
elif config['site'] == 'MT':
	dates = [2019, 2020]
	datasetClass = MTMultipleDates
elif config['site'] == 'MS':
	dates = [2019, 2020]
	datasetClass = MSMultipleDates
elif config['site'] == 'PI':
	dates = [2019, 2020]
	datasetClass = PIMultipleDates
else:
	raise ValueError('Invalid site')

dataset = datasetClass(config = config,
			   dates = dates, 
	 	addPastDeforestationInput = config['addPastDeforestationInput'],
		borderBuffer = 2)


ic(dates)

if issubclass(type(dataset), MultipleDates):
	patchesHandler = PatchesHandlerMultipleDates(
		dataset)
elif config['mode'] == 'evidential2':
	patchesHandler = PatchesHandlerEvidential2()
else:
	patchesHandler = PatchesHandler()


ic| list(self.date_ids): [0]
ic| self.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8]]
ic| dates: [2019, 2020]
ic| self.input_image_shape: 9
ic| self.dataset.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8]]


In [ ]:
manager_class

src.manager.multioutput.ManagerEvidential2

In [ ]:
exp

41

In [ ]:
repetition_ids = manager_class(config, dataset, patchesHandler, logger).get_repetition_ids(exp)  
repetition_n = len(repetition_ids)
print("repetition_ids {}, repetition_n {}".format(repetition_ids, repetition_n))


repetition_ids [0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 3, 4, 5, 6, 7, 8, 9], repetition_n 20


In [ ]:
config['start_repetition_id']

0

In [ ]:
if config['inferring'] == True:
    results = []
    error_count = 0

    # while idx < len(repetition_n):
    # for idx in range(config['start_repetition_id'], repetition_n): # 10 repetitions
    for idx in range(config['start_repetition_id'], config['end_repetition_id']): # 10 repetitions

    # for idx in range(0, 4): # 10 repetitions


        print("Beginning run number {}".format(idx))
        logger = Logger()
        manager = manager_class(config, dataset, patchesHandler, logger)

        print("manager.config", manager.config)

        manager.defineExperiment(exp) # fixed
        manager.defineRepetitionId(idx) # varying from 0 to 10
        manager.setExperimentPath()
        manager.createLogFolders()

        manager.loadDataset()

        # %%
        if config['get_multiple_metrics'] == False:
            predictor_fcn = manager.run_predictor
        else:
            predictor_fcn = manager.run_predictor_multiple_metrics
        result = predictor_fcn()
        print("Result idx {}: {}".format(idx, result))
        results.append(result)

        with open(manager.default_log_name, 'wb') as f:
            pickle.dump(results, f)
        print("Grid execution idx: {}".format(idx))
        del manager
        gc.collect()
        # idx += 1
    

Beginning run number 0
manager.config {'training': True, 'inferring': True, 'site': 'MS', 'training_date': 'current', 'mode': 'evidential2', 'training_times': 20, 'uncertainty_method': 'pred_entropy', 'loadInference': False, 'removePolygons': True, 'plotLandsat': False, 'plotPOI': False, 'training_experiment_id': 41, 'get_multiple_metrics': False, 'start_repetition_id': 0, 'end_repetition_id': 20, 'save_probabilities': False, 'addPastDeforestationInput': True, 'classes_mode': True, 'learning_rate': 0.0001, 'use_cloud_mask': False, 'mask_out_non_biome': False, 'log_filename': 'log.pkl', 'inference_times': 1, 'dropout_training': False}
D:/Jorge/datasets/deforestation/MS/deforestation_past_years.tif
Loaded deforestation past years


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2002, 2004, 2006, 2008, 2010, 2012, 2013, 2014, 2015, 2016,
                                                                    2017, 2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([334024764,   8717093,   6541094,   3824922,   2096535,   1305948,
                                                                      1003721,    903906,    318788,    701575,    310887,    440417,
                                                                       316180,    560438,    600838,    776899], dtype=int64))


Label where deforestation past years is actual date (2020) = 1


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2002, 2004, 2006, 2008, 2010, 2012, 2013, 2014, 2015, 2016,
                                                                    2017, 2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([334024764,   8717093,   6541094,   3824922,   2096535,   1305948,
                                                                      1003721,    903906,    318788,    701575,    310887,    440417,
                                                                       316180,    560438,    600838,    776899], dtype=int64))


Past deforestation different from 0 (no deforestation)


...

[0.81494529 0.94632734 0.82544601 0.03000622 0.27858953 0.43369263
 0.05328917]
threshold:  0.37665326381523984
threshold 0.37665326381523984


ic| label_current_deforestation_test_classified_incorrect.shape: (1105326,)
    predicted_test_classified_incorrect.shape: (1105326,)
ic| TP_H + FN_H + FP_H + TN_H: 1105326
    len(label_mask_current_deforestation_test): 36846296
ic| TP_H + FN_H + FP_H + TN_H: 1105326
    len(label_mask_current_deforestation_test): 36846296


[0.81490364 0.94632734 0.82544601 0.02999829 0.27866206 0.43369263
 0.05328897]
threshold 0.37665326381523984
(36846296,) (36846296,)


ic| self.m_optimal: {'AA': array([0.02999829]),
                     'UEO': array([0.05328897]),
                     'f1': 79.65,
                     'f1_H': array([0.3393076]),
                     'f1_L': array([0.87571205]),
                     'precision_H': array([0.27866206]),
                     'precision_L': array([0.81490364]),
                     'recall_H': array([0.43369263]),
                     'recall_L': array([0.94632734]),
                     'recall_Ltotal': array([0.82544601])}
ic| self.m_audited_optimal: {'f1': array([0.89054495]),
                             'precision': array([0.83563148]),
                             'recall': array([0.95318334])}


cm_audited [[36502173    54333]
 [   13567   276223]]
[0.83563148 0.95318334]
Result idx 19: {'uncertainty_result': {'metrics': {'precision_L': array([0.81490364]), 'recall_L': array([0.94632734]), 'recall_Ltotal': array([0.82544601]), 'AA': array([0.02999829]), 'precision_H': array([0.27866206]), 'recall_H': array([0.43369263]), 'UEO': array([0.05328897]), 'f1_L': array([0.87571205]), 'f1_H': array([0.3393076]), 'f1': 79.65}, 'metrics_audited': {'precision': array([0.83563148]), 'recall': array([0.95318334]), 'f1': array([0.89054495])}, 'exp': 41}}
Grid execution idx: 19


In [ ]:
error_count

0

In [ ]:
print(results)

[{'uncertainty_result': {'metrics': {'precision_L': array([0.94180393]), 'recall_L': array([0.94189787]), 'recall_Ltotal': array([0.78328622]), 'AA': array([0.02999771]), 'precision_H': array([0.4996835]), 'recall_H': array([0.72903583]), 'UEO': array([0.04303796]), 'f1_L': array([0.9418509]), 'f1_H': array([0.59295426]), 'f1': 87.23}, 'metrics_audited': {'precision': array([0.95160309]), 'recall': array([0.95168202]), 'f1': array([0.95164256])}, 'exp': 41}}, {'uncertainty_result': {'metrics': {'precision_L': array([0.98164181]), 'recall_L': array([0.9443531]), 'recall_Ltotal': array([0.76056992]), 'AA': array([0.02999691]), 'precision_H': array([0.78797191]), 'recall_H': array([0.68967544]), 'UEO': array([0.02480493]), 'f1_L': array([0.96263649]), 'f1_H': array([0.73555423]), 'f1': 92.0}, 'metrics_audited': {'precision': array([0.9853273]), 'recall': array([0.95518269]), 'f1': array([0.97002086])}, 'exp': 41}}, {'uncertainty_result': {'metrics': {'precision_L': array([0.91508893]), 'r

In [ ]:
print(len(results))

20


In [ ]:
results[7]

{'uncertainty_result': {'metrics': {'precision_L': array([0.95336232]),
   'recall_L': array([0.94277114]),
   'recall_Ltotal': array([0.85146072]),
   'AA': array([0.03000507]),
   'precision_H': array([0.4334014]),
   'recall_H': array([0.39263607]),
   'UEO': array([0.02781169]),
   'f1_L': array([0.94803715]),
   'f1_H': array([0.41201284]),
   'f1': 89.81},
  'metrics_audited': {'precision': array([0.95792511]),
   'recall': array([0.94831394]),
   'f1': array([0.95309529])},
  'exp': 41}}